In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import VarianceThreshold 
#feature의 variance가 적다는 말은 정보가 별로 없다는 말일 수도 있다. 100개의 샘플의 데이터가 99개 1 1개 0이면
#별로 배울게 없는 데이터이다.
from sklearn.feature_selection import SelectFromModel
#모델의 중요도에 따라서 선택하는 것.
#어떤 특성은 0으로 예측 했을 때 중요한 특성이 1로 하니까 중요도가 안 좋아지는 경우 ==> 중요한 특성
#그런데 0으로 하든 1로 하든 중요도가 변함이 없는 경우 ==> 안중요
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_rows', 100)

In [ ]:
DEBUG = True

In [ ]:
if DEBUG :
    NROWS = 500000
else :
    NROWS = None

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')


In [ ]:
train.shape

### 읽을 데이터가 너무 크다!
* pd.read_csv('....', nrows=integer) 또는
* sample의 fraction을 이용하는 것도 한 방법(random성 있음)
* imbalance data가 걱정 되는 경우 StratifiedSampling 이용


In [ ]:
train = pd.read_csv('../input/train.csv')
train = train.sample(frac=0.1)
train.shape

### Stratified Sampling 깔끔하게 하기 꿀팁!

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
fold = StratifiedKFold(n_splits=10, random_state =42)

In [ ]:
# index뽑기
for trn_idx, tst_idx in fold.split(train, train['target']):
    break

In [ ]:
trn_idx

In [ ]:
train.iloc[trn_idx]

In [ ]:
test.iloc[tst_idx]

깔끔!

In [ ]:
cat_cols = [col for col in train.columns if 'cat' in col]

In [ ]:
cat_cols

In [ ]:
train[cat_cols[0]].value_counts()

In [ ]:
# category의 유니크한 개수
for col in cat_cols:
    print(col,':', train[col].nunique() )
    #high cardinality : ps_cal_11_cat ==> 104개 
    #(one-hot-encoding하게 되면 머리아파짐,, 다른 encoding방법이 필요)

In [ ]:
train.shape

In [ ]:
train = train.drop_duplicates()
train.shape
#중복된 값 없음

In [ ]:
test = test.drop_duplicates()
test.shape

dummy feature에 one_hot_encoding을 실시할 것

In [ ]:
train.info() #object는 없다 --> label_encoding 필요 없음

### Metadata
To facilitate the data management, we'll store meta-inforamtation about the variablel in a DataFrame. This will be helpful when we wnat to select specific variables for analysis, visualization, modeling,...

Concretely we will store:
* role : input, ID, target
* level : nominal, interval, ordinal, binary
* keep : True, False
* dtype : int, float, str


In [ ]:
data = []
for f in train.columns:
    if f == 'target' :
         role = 'target'
    elif f == 'id':
        role = 'id'
    else : 
        role = 'input'
        
    #Defining the level
    if 'bin' in f or f == 'target':
        level =  'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    #Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    #Defining the data type    
    dtype = train[f].dtype
    #Crearting a Dict that contains all the metadata for the variable
    f_dict = {
        'varname':f,
        'role':role,
        'level':level,
        'keep': keep,
        'dtype':dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns = ['varname','role','level','keep','dtype'])
meta.set_index('varname',inplace=True)

In [ ]:
meta

Example to extract all nominal variables that are not dropped

In [ ]:
meta[(meta.level == 'nominal') & (meta.keep)].index

Below the number of variables per role and level are displayed

In [ ]:
pd.DataFrame({'count': meta.groupby(['role','level'])
             ['role'].size()}).reset_index()

정리를 이쁘게 했다.

### Descriptive Statistics
We can also apply the describe method on the dataframe. However, it doesn't make much sense to calculate the mean, std,... on categorical variables and the id variable. We'll explore the categorical variables visually later.

Thanks to our meta file we can easily select the variables on which we want to compute the descriptive statistics. To keep things clear, we'll do this per data type.

Interval variables

In [ ]:
categorical_feat = [col for col in train.columns if 'cat' in col]

In [ ]:
categorical_feat #이런식으로 뽑는 것도 가능하다

In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
train[v].describe()

reg variables
* only ps_reg_03 has missing values
* the range(min to max) differs between the variables. We could apply scaling(e.g StandardScaler), but it depends on the classifier we will want to use.
* linear regression일 경우, feature들 간의 distribution이 너무 다르, 특정 var에 쏠리는 경우가 발생함. StandardScaler사용.(선형모델)
* Tree 모델을 사용할 경우, 절대값의 크기에 영향 별로 안받음. 굳이 안해도 됨

car variables
* ps_car_12 and ps_car_15 have missing values
* again, the range differs and we could apply scaling.

calc variables
* no missing values
* this seems to be some kind of ratio as the maximum is 0.9
* all three `_calc` variables have very similar distributions

__Overall,__ we can see that the range of the interval variables is rather small. Perhaps some transformation(e.g log) is already applied in order to anoymize the data?

Ordinal variables





In [ ]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
train[v].describe()

* Only one missing variable: ps_car_11
* We could apply sacling to deal with the different ranges

Binary variables

In [ ]:
v = meta[(meta.level == 'binary') & (meta.keep)].index
train[v].describe()

- A priori in the train data is 3.645%, which is __strongly imbalanced.__
- From the means we can conclude that for most variables the value is zero in most cases. (대부분이 보험 청구가 안되었다.)

### Handling imbalanced classes
As we mentioned above the proportion of records with target = 1 is far less than target=0. This lead to a model that has great accuracy but does have any added value in practice. Two possible strategies to deal with this problem are:
- oversampling records with target=1
- undersampling records with target=0
There are many more strategies of course and MachineLearningMastery.com gives a nice overview. As we have a rather large training set, se can go for __undersamplling.__

얼마나 1을 잘 맞추느냐가 중요한 상황. gini나 ROC AUC를 사용하는 이유.  
undersampling 많은 걸 줄임, target이 0인 걸 줄일 것


SMOTE!

In [ ]:
desired_apriori = 0.10

#Get the indices per target value
idx_0 = train[train.target ==0].index
idx_1 = train[train.target == 1].index

#Get original number of records per target value
nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

# Calculate the undersampling rate and resulting number of records with target =0
undersampling_rate = ((1-desired_apriori)*nb_1) / (nb_0*desired_apriori)
undersampled_nb_0 = int(undersampling_rate*nb_0)
print('Rate to undersample records with target=0: {}'.format(undersampling_rate))
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

#Randomly select records with target=0 to get at the desired a priori
undersampled_idx = shuffle(idx_0, random_state=37, n_samples = undersampled_nb_0)

#construct list with remaining indices
idx_list = list(undersampled_idx) + list(idx_1)

#return undersampled data frame
train = train.loc[idx_list].reset_index(drop=True) #shuffle해줬으니까 순서대로

In [ ]:
nb_0

In [ ]:
nb_1

### Data Quality Checks

* Checking missing values  
Missings are represeted as -1

In [ ]:
vars_with_missing = []
for f in train.columns :
    missings = train[train[f] == -1][f].count()
    if missings > 0 :
        vars_with_missing.append(f)
        missings_perc = missings/train.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))
        
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))


- ps_car_03_cat and ps_car_05_cat have a large proportion of recoreds with missing values. Remove these variables.
- For the other categorical variables with missing values, we can leave the missing value -1 as such.
- ps_reg_03(continuois) has missing values for 18% of all records. Replace by the mean
- ps_car_11(ordinal) has only 5 records with missing values. Replace by the mode.
- ps_car_12(continuous)has only 1records with missing value. Replace by the mean.
- ps_car_14(continuous) has missing values for 7% of all records. Replace by the mean.

### Missing Value는 함부로 채우면 안된다
모델을 학습시켜서 채우는 방법도 있다.

가끔씩 missing value값이 확 뛸 때가 있음. 이경우는 missing value에 정보가 있다는 말일 수 도 있음. 하지만 porto에서는 정보가 없다.

In [ ]:
#Dropping the variables with too many missing values
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
train.drop(vars_to_drop, inplace=True, axis=1)
meta.loc[(vars_to_drop), 'keep'] = False

In [ ]:
#Imputing with the mean or mode
mean_imp = Imputer(missing_values=-1, strategy='mean', axis=0)
mode_imp = Imputer(missing_values=-1, strategy='most_frequent', axis=0)
train['ps_reg_03'] = mean_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_reg_car_12'] = mean_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = mean_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = mean_imp.fit_transform(train[['ps_car_11']]).ravel()

In [ ]:
mean_imp.fit_transform(train[['ps_car_14']]).shape

### Checing the cardinality of the categorical variables
Cardinality refers to the number of different values in a variable. As we will create dummy variables from the categorical variables later on, we need to check whether there are variables with many distinct values. We should handle these variables differently as they would result in many dummy variables.

In [ ]:
v = meta[(meta.level =='nominal') & (meta.keep)].index
for f in v:
    dist_values = train[f].value_counts().shape[0]
    print('Varaible {} has {} distinct values'.format(f, dist_values))

In [ ]:
train[v[0]].value_counts().shape[0]

In [ ]:
train[v[0]].nunique() # 같은 결과

One hot Encoding 하는데 Cardinality가 높다면, 많은 distinct values들이 있다면, 1000개의 칼럼 계산할게 너무 많아짐.

Only ps_car_11 has many distinct values, although it is still reasonable.

EDIT: nickycan made an excellent remark on the fact that my first solution could lead to data leakage. He also pointed me to another kernel made by oliver which deals with that. I therefore replaced this part with the kernel of oliver. All credits go to him. It is so great what you can learn by participating in the Kaggle competitions :)

74개의 칼럼이 생기게 되는데...Accept할지 말지는 알아서 결정.

칼럼이 너무 많아지면, 트리모델을 사용할 경우, 트리의 깊이도 깊어지고, 중요한 질문을 못 건드릴 가능성 높아짐, 답으로 가기 위한 과정에서 멀어짐, 원하지 않는 split이 생길 수 있음. one hot encoding많아지면 힘들다... ==> mean encoding(Overfitting 가능성, 그래서 noise를 추가해줌) OR Frequency Encoding 사용한다.

## Mean Encoding

In [ ]:
# Script by https://www.kaggle.com/ogrellier
# Code: https://www.kaggle.com/ogrellier/python-target-encoding-for-categorical-features
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [ ]:
trn_series = train["ps_car_11_cat"]
tst_series = test["ps_car_11_cat"]
target = train.target
min_samples_leaf = 100
smoothing = 10
noise_level = 0.01

In [ ]:
train_encoded, test_encoded = target_encode(train["ps_car_11_cat"], 
                             test["ps_car_11_cat"], 
                             target=train.target, 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
    
train['ps_car_11_cat_te'] = train_encoded
train.drop('ps_car_11_cat', axis=1, inplace=True)
meta.loc['ps_car_11_cat','keep'] = False  # Updating the meta
test['ps_car_11_cat_te'] = test_encoded
test.drop('ps_car_11_cat', axis=1, inplace=True)

In [ ]:
temp = pd.concat([trn_series, target], axis=1)

In [ ]:
temp.groupby(by = trn_series.name)[target.name].agg(['mean', 'count'])

In [ ]:
averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])

In [ ]:
# distribution smoothing
smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))

In [ ]:
smoothing

In [ ]:
prior = target.mean()

In [ ]:
prior

In [ ]:
averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing

In [ ]:
averages

In [ ]:
ft_trn_series = pd.merge(trn_series.to_frame(trn_series.name), 
                        averages.reset_index().rename(columns = 
                            {'index':target.name, target.name : 'average'}))

In [ ]:
averages #target의 mean을 smoothing한 정보만 남는다.

In [ ]:
trn_series.to_frame(trn_series.name), averages.reset_index()

In [ ]:
averages.reset_index().rename(columns={'index' : target.name, target.name : 'average'})

In [ ]:
pd.merge(trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns = {'index':target.name,
                                                target.name:'average'})
,on = tst_series.name, how = 'left')['average'].rename(trn_series.name + '_mean').fillna(prior)    
#left는 꼭 해줄 것, 안해주면 샘플이 빠지는 경우가 발생

In [ ]:
ft_trn_series.index = trn_series.index

In [ ]:
#노이즈 추가, 너무 discrete하지 않게 continuous하게 만들어준다.
add_noise

In [ ]:
0.01 * np.random.randn(ft_trn_series.shape[0])

In [ ]:
train['ps_car_11_cat_te'] = train_encoded

In [ ]:
train

In [ ]:
target

### Exploratory Data Visualization

#### Categorical variables
Lets' look into the categorical varaibles and the proportion of customers with target=1

In [ ]:
v = meta[(meta.level == 'nominal')& (meta.keep)].index

In [ ]:
sns.set(font_scale=2)

In [ ]:
sns.set(font_scale=2)
for f in v:
    plt.figure()
    fig, ax = plt.subplots(figsize=(20,10))
    # Calculate the percentage of target=1 per category value
    cat_perc = train[[f, 'target']].groupby([f],as_index=False).mean()
    cat_perc.sort_values(by='target', ascending=False, inplace=True)
    # Bar plot
    # Order the bars descending on target mean
    sns.barplot(ax=ax, x=f, y='target', data=cat_perc, order=cat_perc[f])
    plt.ylabel('% target', fontsize=18)
    plt.xlabel(f, fontsize=18)
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.show();


As we can see from the variables with missing values, it is a good idea to keep the missing values as a separate category value, instead of replacing them by the mode for instance. The customers with a missing value appear to have a much higher (in some cases much lower) probability to ask for an insurance claim.

#### Interval variables
Checking the correlations between interval variables. A heatmap is a good way to visualize the correlation between variables. The code below is based on an example by Michael Waskom

In [ ]:
sns.set(font_scale=1)

In [ ]:
def corr_heatmap(v):
    correlations = train[v].corr()

    # Create color map ranging between two colors
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .75})
    plt.show();
    
v = meta[(meta.level == 'interval') & (meta.keep)].index
corr_heatmap(v)

There are a strong correlations between the variables:

- ps_reg_02 and ps_reg_03 (0.7)
- ps_car_12 and ps_car13 (0.67)
- ps_car_12 and ps_car14 (0.58)
- ps_car_13 and ps_car15 (0.67)

Seaborn has some handy plots to visualize the (linear) relationship between variables. We could use a pairplot to visualize the relationship between the variables. But because the heatmap already showed the limited number of correlated variables, we'll look at each of the highly correlated variables separately.
NOTE: I take a sample of the train data to speed up the process.



In [ ]:
s = train.sample(frac=0.1)

ps_reg_02 and ps_reg_03
As the regression line shows, there is a linear relationship between these variables. Thanks to the hue parameter we can see that the regression lines for target=0 and target=1 are the same.

In [ ]:
sns.lmplot(x='ps_reg_02', y='ps_reg_03', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

ps_car_12 and ps_car_13

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_13', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

ps_car_12 and ps_car_14

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_14', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

ps_car_13 and ps_car_15¶

In [ ]:
sns.lmplot(x='ps_car_15', y='ps_car_13', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

Allright, so now what? How can we decide which of the correlated variables to keep? We could perform Principal Component Analysis (PCA) on the variables to reduce the dimensions. In the AllState Claims Severity Competition I made this kernel to do that. But as the number of correlated variables is rather low, we will let the model do the heavy-lifting.

Checking the correlations between ordinal variables

In [ ]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
corr_heatmap(v)

For the ordinal variables we do not see many correlations. We could, on the other hand, look at how the distributions are when grouping by the target value.

단순히 숫자만 뽑을 것이 아니라, 숫자를 믿지 말고 일일이 다시 그림을 그려보면서 분포에 맞는 값이 나온 건지 재확인 해봐야한다.(진짜 데이터포인트가 선형으로 되어있는지 아님 선형적인게 없는데 높은건지 확인을해야된다. 수치에 속지 말자)

### Feature engineering

Creating dummy variables
The values of the categorical variables do not represent any order or magnitude. For instance, category 2 is not twice the value of category 1. Therefore we can create dummy variables to deal with that. We drop the first dummy variable as this information can be derived from the other dummy variables generated for the categories of the original variable.

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index
print('Before dummification we have {} variables in train'.format(train.shape[1]))
train = pd.get_dummies(train, columns=v, drop_first=True)
print('After dummification we have {} variables in train'.format(train.shape[1]))

So, creating dummy variables adds 52 variables to the training set.

### Creating interaction variables

In [ ]:
v = meta[(meta.level =='interval') & (meta.keep)].index
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
# interaction_only=True면 자신 제곱하는 것 제외됨, 
interactions = pd.DataFrame(data = poly.fit_transform(train[v]), columns = poly.get_feature_names(v))
interactions.drop(v, axis=1, inplace=True)
print('Before creating interactions we have {} variables in train'.format(train.shape[1]))
train = pd.concat([train, interactions], axis=1)
print('After creating interactions we have {} variables in train'.format(train.shape[1]))

In [ ]:
poly.fit_transform(train[v])

In [ ]:
poly.get_feature_names(v)

In [ ]:
interactions #많은 feature들이 생긴 것을 알 수 있다.

This adds extra interaction variables to the train data. Thanks to the get_feature_names method we can assign column names to these new variables.

### Feature Selection

* Removing features with low or zero variance  
Personally, I prefer to let the classifier algorithm chose which features to keep. But there is one thing that we can do ourselves. That is removing features with no or a very low variance. Sklearn has a handy method to do that: VarianceThreshold. By default it removes features with zero variance. This will not be applicable for this competition as we saw there are no zero-variance variables in the previous steps. But if we would remove features with less than 1% variance, we would remove 31 variables.

In [ ]:
selector = VarianceThreshold(threshold=.01)
selector.fit(train.drop(['id','target'],axis=1))

In [ ]:
f = np.vectorize(lambda x : not x)
v = train.drop(['id','target'], axis=1).columns[f(selector.get_support())]
print('{} variables have too low variance.'.format(len(v)))
print('These variables are {}'.format(list(v)))

In [ ]:
selector.get_support()

In [ ]:
f(selector.get_support()) #위 값을 뒤집어 준다.

In [ ]:
~selector.get_support() #이것도 가능

We would lose rather many variables if we would select based on variance. But because we do not have so many variables, we'll let the classifier chose. For data sets with many more variables this could reduce the processing time.

Sklearn also comes with other feature selection methods. One of these methods is SelectFromModel in which you let another classifier select the best features and continue with these. Below I'll show you how to do that with a Random Forest

### Selecting features with a Random Forest and SelectFromModel
Here we'll base feature selection on the feature importances of a random forest. With Sklearn's SelectFromModel you can then specify how many variables you want to keep. You can set a threshold on the level of feature importance manually. But we'll simply select the top 50% best variables.

The code in the cell below is borrowed from the GitHub repo of Sebastian Raschka. This repo contains code samples of his book Python Machine Learning, which is an absolute must to read.

### 1000개의 피쳐를 일일이 성능보고 셀렉션 하면 너무 오래걸림
- 블록을 사용하자
- 10개씩 20개씩 묶어서 사용


- 20개의 모델 베이스라인을 만들고
- +random choosing 20
- 40개의 모델 학 -> 베이스모델과 비교
- if 성능 향상 -> feature importance 상위 10%에 + 새로 추가된 거 생기면
- 걔를 남기고 
- 안생기면 random choosing, 반복

In [ ]:
X_train = train.drop(['id','target'], axis=1)
y_train = train['target']

In [ ]:
feat_labels = X_train.columns

In [ ]:
rf = RandomForestClassifier(n_estimators=20, random_state=0, n_jobs=-1)

In [ ]:
rf.fit(X_train[:2000],y_train[:2000])

In [ ]:
rf.feature_importances_ #갯수가 너무 적어서 안나온다

In [ ]:
rf.get_params()

In [ ]:
indices = np.argsort(rf.feature_importances_)[::-1]

In [ ]:
indices

In [ ]:
importances = rf.feature_importances_

In [ ]:
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,feat_labels[indices[f]], importances[indices[f]]))


with SelectFromModel we can specify which prefit classifier to use and what the threshold is for the feature importances. With the get_support method we can then limit the number of variables in the train data.

In [ ]:
sfm = SelectFromModel(rf, threshold='median', prefit=True)
print('Number of features before selection: {}'.format(X_train.shape[1]))
n_features = sfm.transform(X_train).shape[1]
print('Number of features after selection: {}'.format(n_features))
selected_vars = list(feat_labels[sfm.get_support()])

In [ ]:
train = train[selected_vars + ['target']] 

### Feature Scaling
As mentioned before, we can apply standard scaling to the training data. Some classifiers perform better when this is done.

In [ ]:
scaler = StandardScaler()
scaler.fit_transform(train.drop(['target'],axis=1))

### 주의!
scaler.fit_trainform(train)  
scaler.transform(test) 이렇게 해주어야 함.  
훈련셋으로 훈련시키고 테스트셋에 적용시키기